In [2]:
import pandas as pd
import numpy as np

In [3]:
train_articles = pd.read_csv("/data/semeval-2020/task-11/processed/train_article.csv")

In [4]:
spans = pd.read_csv("/data/semeval-2020/task-11/processed/span_labels.csv")

In [5]:
train_articles.head()

,article_id,content
0,762956953,Iran Admits To Aiding Al-Qaeda and Facilitatin...
1,787529309,The Last-Minute Character Assassination of Jud...
2,999001296,Altered Election Documents Tied To Florida Dem...
3,729348908,Virginia man who wanted to join ISIS pleads gu...
4,111111134,Paul Manafort Secretly Met With Julian Assange...


In [6]:
spans.head()

,article_id,start,end,label
0,776616374,149,174,Exaggeration_Minimisation
1,776616374,751,786,Exaggeration_Minimisation
2,776616374,831,849,Name_Calling_Labeling
3,776616374,2641,2650,Flag-Waving
4,776616374,2771,2782,Loaded_Language


In [7]:
spans["start_end"] = spans.apply(lambda x:(x.start,x.end,x.label),axis=1)

In [8]:
spans.head()

,article_id,start,end,label,start_end
0,776616374,149,174,Exaggeration_Minimisation,"(149, 174, Exaggeration_Minimisation)"
1,776616374,751,786,Exaggeration_Minimisation,"(751, 786, Exaggeration_Minimisation)"
2,776616374,831,849,Name_Calling_Labeling,"(831, 849, Name_Calling_Labeling)"
3,776616374,2641,2650,Flag-Waving,"(2641, 2650, Flag-Waving)"
4,776616374,2771,2782,Loaded_Language,"(2771, 2782, Loaded_Language)"


In [9]:
article_with_span = train_articles.merge(spans,on="article_id")

In [10]:
article_with_span["article_length"] = article_with_span.content.apply(lambda x: len(x.split(" ")))

In [11]:
article_with_span.head(100)

,article_id,content,start,end,label,start_end,article_length
0,762956953,Iran Admits To Aiding Al-Qaeda and Facilitatin...,630,669,Exaggeration_Minimisation,"(630, 669, Exaggeration_Minimisation)",392
1,787529309,The Last-Minute Character Assassination of Jud...,70,88,Loaded_Language,"(70, 88, Loaded_Language)",1568
2,787529309,The Last-Minute Character Assassination of Jud...,288,320,Loaded_Language,"(288, 320, Loaded_Language)",1568
3,787529309,The Last-Minute Character Assassination of Jud...,924,961,Loaded_Language,"(924, 961, Loaded_Language)",1568
4,787529309,The Last-Minute Character Assassination of Jud...,2405,2431,Loaded_Language,"(2405, 2431, Loaded_Language)",1568
...,...,...,...,...,...,...,...
95,789121265,Juanita Broaddrick: Senator Feinstein Had No I...,1969,1979,Loaded_Language,"(1969, 1979, Loaded_Language)",710
96,789121265,Juanita Broaddrick: Senator Feinstein Had No I...,2018,2195,Whataboutism_Straw_Men_Red_Herring,"(2018, 2195, Whataboutism_Straw_Men_Red_Herring)",710
97,789121265,Juanita Broaddrick: Senator Feinstein Had No I...,2566,2577,Loaded_Language,"(2566, 2577, Loaded_Language)",710
98,789121265,Juanita Broaddrick: Senator Feinstein Had No I...,2848,2893,Doubt,"(2848, 2893, Doubt)",710


In [12]:
def create_label_seq(row):
    article_id = row.article_id
    if (article_id == 762956953):
        return []
    article = row.content
    print(article_id)
    len_article = len(article.split(" "))
    
    article_labels_seq = []
    article_span_seq = []
    article_no_propaganda_seq = []
    
    for i in range(len_article):
        article_labels_seq.append("O")
    
    spans_for_article = list(set(spans[spans["article_id"] == article_id].start_end.values.tolist()))
    print(spans_for_article)
    for start_end in spans_for_article:
        start = start_end[0]
        end = start_end[1]
        propaganda = start_end[2]
        for ix in range(start,end):
            
            
            
            new_tag = None
            print(len(article_labels_seq),start,end)
            current_tag = article_labels_seq[ix]
            if current_tag == "O":
                article_labels_seq[ix] = propaganda
            else:
                current_tags = current_tag.split(",")
                current_tags.append(propaganda)
                current_tag_string = ",".join(current_tags)
                article_labels_seq[ix] = current_tag_string
    
    return article_labels_seq

In [13]:
train_articles["label"] = train_articles.apply(lambda x: create_label_seq(x),axis=1)

787529309
[(2871, 2877, 'Loaded_Language'), (5159, 5195, 'Loaded_Language'), (9173, 9282, 'Doubt'), (288, 320, 'Loaded_Language'), (8200, 8310, 'Causal_Oversimplification'), (5394, 5414, 'Name_Calling_Labeling'), (7626, 7677, 'Loaded_Language'), (16, 40, 'Slogans'), (8337, 8366, 'Name_Calling_Labeling'), (8686, 8711, 'Loaded_Language'), (323, 347, 'Slogans'), (9303, 9384, 'Causal_Oversimplification'), (6056, 6347, 'Doubt'), (8567, 8586, 'Loaded_Language'), (924, 961, 'Loaded_Language'), (8209, 8232, 'Name_Calling_Labeling'), (6678, 6784, 'Appeal_to_Authority'), (70, 88, 'Loaded_Language'), (2338, 2357, 'Name_Calling_Labeling'), (2405, 2431, 'Loaded_Language'), (7814, 7853, 'Slogans'), (2694, 2708, 'Loaded_Language'), (9661, 9697, 'Loaded_Language'), (3158, 3173, 'Loaded_Language')]
1568 2871 2877
787529309
[(2871, 2877, 'Loaded_Language'), (5159, 5195, 'Loaded_Language'), (9173, 9282, 'Doubt'), (288, 320, 'Loaded_Language'), (8200, 8310, 'Causal_Oversimplification'), (5394, 5414, 'Name

IndexError: ('list index out of range', 'occurred at index 1')

In [14]:
len(train_articles[train_articles["article_id"] == 787529309].content[1].split(" "))

1568

In [15]:
spans[spans["article_id"] == 787529309]

,article_id,start,end,label,start_end
1539,787529309,70,88,Loaded_Language,"(70, 88, Loaded_Language)"
1540,787529309,288,320,Loaded_Language,"(288, 320, Loaded_Language)"
1541,787529309,924,961,Loaded_Language,"(924, 961, Loaded_Language)"
1542,787529309,2405,2431,Loaded_Language,"(2405, 2431, Loaded_Language)"
1543,787529309,2694,2708,Loaded_Language,"(2694, 2708, Loaded_Language)"
1544,787529309,3158,3173,Loaded_Language,"(3158, 3173, Loaded_Language)"
1545,787529309,8209,8232,Name_Calling_Labeling,"(8209, 8232, Name_Calling_Labeling)"
1546,787529309,8337,8366,Name_Calling_Labeling,"(8337, 8366, Name_Calling_Labeling)"
1547,787529309,8567,8586,Loaded_Language,"(8567, 8586, Loaded_Language)"
1548,787529309,8686,8711,Loaded_Language,"(8686, 8711, Loaded_Language)"


In [16]:
labels = ["O","O"]
np.array(labels)

array(['O', 'O'], dtype='<U1')

In [17]:
list(range(3,6))

[3, 4, 5]

## Approach 2 

In [22]:
import pandas as pd
import glob
import swifter
path = r'/data/semeval-2020/task-11/datasets/train-tagged_article/'
all_files = glob.glob(path + "/*.txt")

content_tuple = []

for filename in all_files:
    file = open(filename)
    content = "".join(file.readlines()).replace("\n"," ")
    article_id = filename.split("article")[-1].split(".txt")[0]
    content_tuple.append((article_id,content))

In [23]:
content_df = pd.DataFrame(content_tuple,columns=["article_id","content"])

In [24]:
content_df.head(3)

,article_id,content
0,762956953,Iran Admits To Aiding Al-Qaeda and Facilitatin...
1,787529309,The Last-Minute <span-11 Character Assassinati...
2,999001296,Altered Election Documents Tied To Florida Dem...


In [25]:
import re
def get_technique_id(token):
    if "<span" in token:
        results_start_span = re.findall("<span-\d+",token)[0].replace("<span-","")
    elif "-/span>" in token:  
        results_start_span = re.findall("\d+-/span>",token)[0].replace("-/span>","")

    return results_start_span

def get_propaganda_sequence(row):
    content = row.content.replace("><","> <").replace(">“<","> <")\
                         .replace(">(<","> <").replace(">.<","> <")\
                         .replace(">\'<","> <")
    article_id = row.article_id
#     print(article_id)
    label_seq = []
    label_propaganda_seq =[]
    running_token = []
    for token in content.split(" "):
        if "-/span>" in token: 
#             print(running_token)
#             print(get_technique_id(token))
            running_token.remove(get_technique_id(token))
            running_token = sorted(running_token)
        
        if len(running_token) == 0:                        
            label_propaganda_seq.append("O")    
        else:
            label_propaganda_seq.append(",".join(running_token))

        if "<span" in token:
            running_token.append(get_technique_id(token))            
            running_token = sorted(running_token)
            
    new_content = [ re.sub("<span-\d+","",re.sub("\d+-/span>","",token)) for token in content.split(" ")]
    
    zipped_seq = list(zip(new_content,label_propaganda_seq))
    
    seq_to_return = [i for i in zipped_seq if i[0] != ""]
    
    return seq_to_return

In [26]:
get_technique_id("<span-11")

'11'

In [27]:
# get_propaganda_sequence(content_df[content_df["article_id"] == "736231219"].iloc[0].content)

In [28]:
# content_df[content_df["article_id"] == "736231219"].iloc[0].content

In [29]:
# get_propaganda_sequence({"content":content_df.iloc[2].content,"article_id":11})

In [30]:
content_df["tagged_sequence_with_propaganda_types"] = content_df.swifter.apply(get_propaganda_sequence,axis=1)

/home/raghavan/anaconda3/envs/semeval_2020/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [31]:
def propaganda_type_seq_to_simple_yes_or_no_seq(taggings):
    simple_taggings = []
    for i in taggings:
        if i[1] != "O":
            simple_taggings.append((i[0],"I"))
        else:
            simple_taggings.append(i)
    return simple_taggings        

In [32]:
content_df["tagged_sequence"] = content_df.tagged_sequence_with_propaganda_types.swifter.apply(propaganda_type_seq_to_simple_yes_or_no_seq)

In [33]:
for i in zip(content_df.iloc[1]["tagged_sequence_with_propaganda_types"],content_df.iloc[1]["tagged_sequence"]):
    print(i)

(('The', 'O'), ('The', 'O'))
(('Last-Minute', 'O'), ('Last-Minute', 'O'))
(('Character', '11'), ('Character', 'I'))
(('Assassination', '11'), ('Assassination', 'I'))
(('of', 'O'), ('of', 'O'))
(('Judge', 'O'), ('Judge', 'O'))
(('Kavanaugh', 'O'), ('Kavanaugh', 'O'))
(('Using', 'O'), ('Using', 'O'))
(('any', 'O'), ('any', 'O'))
(('despicable', '8'), ('despicable', 'I'))
(('tactic', '8'), ('tactic', 'I'))
(('at', 'O'), ('at', 'O'))
(('hand', 'O'), ('hand', 'O'))
(('to', 'O'), ('to', 'O'))
(('derail', 'O'), ('derail', 'O'))
(('Judge', 'O'), ('Judge', 'O'))
(('Brett', 'O'), ('Brett', 'O'))
(('Kavanaugh’s', 'O'), ('Kavanaugh’s', 'O'))
(('Supreme', 'O'), ('Supreme', 'O'))
(('Court', 'O'), ('Court', 'O'))
(('confirmation', 'O'), ('confirmation', 'O'))
(('less', 'O'), ('less', 'O'))
(('than', 'O'), ('than', 'O'))
(('a', 'O'), ('a', 'O'))
(('week', 'O'), ('week', 'O'))
(('before', 'O'), ('before', 'O'))
(('the', 'O'), ('the', 'O'))
(('Senate', 'O'), ('Senate', 'O'))
(('Judiciary', 'O'), ('Judic

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
train_df , test_df =  train_test_split(content_df,test_size=0.2)

In [36]:
train_df.head(3)

,article_id,content,tagged_sequence_with_propaganda_types,tagged_sequence
3,729348908,Virginia man who wanted to join ISIS pleads gu...,"[(Virginia, O), (man, O), (who, O), (wanted, O...","[(Virginia, O), (man, O), (who, O), (wanted, O..."
290,709732928,Uranium One <span-8 Bombshell 8-/span>: Eviden...,"[(Uranium, O), (One, O), (Bombshell, 8), (:, O...","[(Uranium, O), (One, O), (Bombshell, I), (:, O..."
111,735815173,Dem Congressman Corrects ADL's Claim That He I...,"[(Dem, O), (Congressman, O), (Corrects, O), (A...","[(Dem, O), (Congressman, O), (Corrects, O), (A..."


In [37]:
train_tag_array = train_df.tagged_sequence.values.tolist()
test_tag_array = test_df.tagged_sequence.values.tolist()

In [38]:
# final_string = ""
# for i in train_tag_array:
#     for k in i:
#         final_string = final_string + k[0] + " " + k[1] + "\n"
#     final_string = final_string + "\n"    

KeyboardInterrupt: 

In [ ]:
# with open('train_task.txt','w') as f:
#     f.write(final_string)

In [ ]:
# test_final_string = ""
# for i in test_tag_array:
#     for k in i:
#         test_final_string = final_string + k[0] + " " + k[1] + "\n"
#     test_final_string = final_string + "\n"    

In [ ]:
# with open('dev_task.txt','w') as f:
#     f.write(test_final_string)

In [ ]:
# train_df["article_length"] = train_df.content.swifter.apply(lambda x:len(x.split(" ")))

In [39]:
# max(train_df.article_length.values.tolist())

In [51]:
final_string = ""
for i in train_tag_array:
    for k in i:
        final_string = final_string + k[0] + " " + k[1] + "\n"
        if k[0][-1] == "." or k[0][-1] == "?":
            final_string = final_string + "\n"
    final_string = final_string + "\n"    

In [52]:
with open('train_task.txt','w') as f:
    f.write(final_string)

In [53]:
test_final_string = ""
for i in test_tag_array:
    for k in i:
        test_final_string = test_final_string + k[0] + " " + k[1] + "\n"
        if k[0][-1] == "." or k[0][-1] == "?":
            test_final_string = test_final_string + "\n"
    test_final_string = test_final_string + "\n"    

In [54]:
with open('dev_task.txt','w') as f:
    f.write(test_final_string)

## Prepare test dataset

In [55]:
dev_articles_path = "/data/semeval-2020/task-11/datasets/dev-articles"

In [56]:
import pandas as pd
import glob
import swifter
all_files = glob.glob(dev_articles_path + "/*.txt")

dev_content_tuple = []

for filename in all_files:
    file = open(filename)
    content = "".join(file.readlines()).replace("\n"," ")
    article_id = filename.split("article")[-1].split(".txt")[0]
    dev_content_tuple.append((article_id,content))

In [47]:
dev_content_tuple[0][1].split(". ")

["Unbelievable! Sharia New Mexico: Islamic compound jihadis RELEASED on bond after charges of “Islamophobia” and “racism” (Islam is not a race)  Editor's Note: Talk about injustice! There were remains of a 4-year-old boy found there, who allegedly died while they were performing some sort of Islamic ritual over him",
 'The Bundys and their supporters got nearly two years in jail and no one hurt a single person! Shame on those who are supposed to uphold law and justice! Shame! Taos County Sheriff Jerry Hogrefe testified that they found children holding boxes of ammunition, and that one child was found with a gun…',
 'Because sharia trumps dead children, school shooting training, kidnapping and jihad training',
 'The children discovered at an “extremist Muslim” compound in New Mexico earlier this month were both trained to use firearms and taught multiple tactical techniques in order to kill teachers, law enforcement and other institution…',
 'state prosecutors said on Monday',
 'take ou

In [48]:
article_id_sentence = [(i[0],k) for i in dev_content_tuple for k in i[1].split(". ")]

In [49]:
dev_article_text = ""
for combo in article_id_sentence:
    for token in combo[1].split(" "):
        dev_article_text = dev_article_text + token + "\n"
    dev_article_text = dev_article_text + "\n"    

In [50]:
with open('test_task.txt',"w") as f:
    f.write(dev_article_text)